# Demo: Discover gradual magnitude pruning for small language model training

In-training pruning integrates the pruning process directly into the model's training or fine-tuning phase. This allows the model to adapt to the removal of weights, often leading to better performance recovery compared to post-training pruning, especially when higher sparsity levels are desired.

> **Overview**: We'll apply gradual magnitude pruning during fine-tuning of a pre-trained language model. This technique removes weights progressively while the model learns to compensate for their absence.
> 
> **Goal**: Learn how to implement iterative pruning during training, allowing models to co-adapt to sparsity and task requirements simultaneously—achieving better compression-accuracy tradeoffs than post-training methods.
> 
> **Scenario**: Imagine that you are developing a customer feedback analysis system for a retail chain (same as Lesson 2). The system processes product reviews and social media mentions to extract sentiment in real-time across thousands of stores. Your current DistilBERT model performs excellently on the cloud, but you need now to expand to edge devices in 10,000 stores! And, you are running in two critical issues:
> <br> - _Limited memory on store kiosks_
> <br> - _Processors with sparse tensors support but no GPU acceleration on store kiosks_
> 
> Post-training pruning achieved only 84% accuracy at 30% sparsity—below your 90% threshold. Since you have access to customer review data and can afford retraining time, gradual magnitude pruning during fine-tuning offers a better path. By carefully ramping up sparsity during training, the model maintains 92% accuracy even at 50% sparsity, meeting both efficiency and accuracy requirements.
> 
> **Tools**: PyTorch, Hugging Face Transformers, Evaluate, Datasets, NumPy, Matplotlib

## Step 1: Setup
Let's begin by importing necessary libraries and setting up our environment with better configuration for stable pruning.

In [1]:
# # Uncomment to install necessary libraries, then comment out the cell block again and restart the notebook
# !pip install transformers datasets torch torchvision torchaudio accelerate evaluate scikit-learn matplotlib tqdm

In [2]:
# Import necessary libraries 
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from copy import deepcopy

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers.optimization import get_cosine_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset
import evaluate

from tqdm.auto import tqdm

# For neural network pruning with PyTorch
import torch.nn.utils.prune as prune

# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create output directory
output_dir = "assets/demo1"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print("Setup complete!")

/home/student/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Setup complete!


> **Environment setup**: This demo uses Hugging Face's ecosystem for transformer models (`transformers`), dataset handling (`datasets`), and evaluation metrics (`evaluate`). PyTorch provides the underlying neural network operations and pruning utilities. Finally, the `accelerate` library can be used to ensure efficient training even on limited hardware.
> <br> Also, we set deterministic seeds to ensure reproducible results across runs and device detection ensures we use GPU acceleration when available, which is crucial for transformer fine-tuning.

## Step 2: Load the model, tokenizer, and dataset

We'll use a smaller pre-trained model like DistilBERT fine-tuned on a sentiment analysis task (e.g., SST-2) as a starting point. This is the same as for Lesson 2's pruning exercise, but with a more carefully curated training dataset.

In [3]:
# Load pre-trained model and tokenizer
model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load the dataset (SST-2 for sentiment analysis)
raw_datasets = load_dataset("glue", "sst2")

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Create moderate-sized datasets for better pruning stability
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))  # Larger than before
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(400))

print(f"Model: {model_checkpoint}")
print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Map: 100%|██████████| 1821/1821 [00:00<00:00, 9976.65 examples/s] 


Model: distilbert-base-uncased-finetuned-sst-2-english
Training samples: 2000
Evaluation samples: 400


> **Data preparation strategy**: We're working with SST-2, a dataset of movie reviews labeled as positive or negative sentiment. For this demo, we use 1,000 training examples to keep things fast—in production, you'd use your full customer review dataset. 
> <br> For the data processing, the AutoTokenizer automatically handles converting text into numbers that our model can process, including special tokens and vocabulary specific to DistilBERT. Then, we pad all sequences to 128 tokens to ensure uniform input sizes for batch processing, and shuffle the data so the model learns general patterns rather than memorizing the order of reviews.
> 
> In real-world deployment, you'd typically use dynamic padding for efficiency and ensure your tokenizer matches the one used during training to maintain consistency.

## Step 3: Establish baseline performance
Let's measure the baseline performance on key metrics: accuracy, model size, and inference speed.

Note that we'll defer the inference speed test until we can run both models back-to-back. This minimizes variability from system resources and gives us a fairer comparison.

In [4]:
# Compute accuracy metrics from predictions
def compute_metrics(eval_pred):
    logits_or_preds, labels = eval_pred

    # Convert to NumPy arrays
    predictions = np.array(logits_or_preds)
    labels = np.array(labels)
    
    # If preds are logits (2D), apply argmax
    if predictions.ndim > 1:
        predictions = np.argmax(logits_or_preds, axis=-1)
    
    # Convert to lists for the evaluate library
    predictions_list = predictions.tolist() if hasattr(predictions, 'tolist') else list(predictions)
    labels_list = labels.tolist() if hasattr(labels, 'tolist') else list(labels)
    
    accuracy = accuracy_metric.compute(predictions=predictions_list, references=labels_list)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions_list, references=labels_list, average="weighted")["f1"]
    return {"accuracy": accuracy, "f1": f1}

# Calculate model size and parameters
def get_model_size_mb(model):
    param_size = sum(p.nelement() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.nelement() * b.element_size() for b in model.buffers())
    return (param_size + buffer_size) / (1024 * 1024)

# Measure inference speed 
def measure_inference_speed(model, dataloader, num_samples=100):
    model.eval()
    times = []
    
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= num_samples:
                break
            
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Warm up
            if i == 0:
                for _ in range(5):
                    _ = model(**batch)
            
            # Time the forward pass
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            
            start_time = time.time()
            _ = model(**batch)
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            end_time = time.time()
            
            times.append(end_time - start_time)
    
    return np.mean(times[1:]), np.std(times[1:])  # Exclude first timing

In [5]:
# Load model
baseline_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
baseline_model = baseline_model.to(device)

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Quick evaluation
training_args = TrainingArguments(
    output_dir=f"{output_dir}/baseline_eval",
    per_device_eval_batch_size=32,
    do_train=False,
    do_eval=True,
    report_to=[]
)

trainer = Trainer(
    model=baseline_model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("Evaluating baseline model...")
baseline_results = trainer.evaluate()
print(f"Baseline metrics: Accuracy={baseline_results['eval_accuracy']:.4f}, F1={baseline_results['eval_f1']:.4f}")

baseline_size = get_model_size_mb(baseline_model)
total_params = sum(p.numel() for p in baseline_model.parameters())
print(f"Baseline model size: {baseline_size:.2f} MB")
print(f"Total parameters: {total_params:,}")

/tmp/ipykernel_76/2086735421.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluating baseline model...


Baseline metrics: Accuracy=0.9025, F1=0.9023
Baseline model size: 255.42 MB
Total parameters: 66,955,010


> **Baseline snapshot**: Our starting point shows solid performance—90.2% accuracy on sentiment analysis, which is quite good for this task! The model weighs in at 255MB with nearly 67 million parameters. This is typical for transformer models: excellent performance but hefty size. 
> 
> For edge devices with 512MB RAM, this would consume half the available memory just for the model weights—leaving little room for the actual application. This is exactly why we need compression techniques like pruning during training!

## Step 4: Configure the pruning strategy

Now, let's define a pruning strategy with reasonable hyperparameters for the small dataset.

In [6]:
# Improved pruning configuration
target_sparsity = 0.5  # 50% sparsity
pruning_frequency = 100
num_epochs = 10
learning_rate = 1e-5
batch_size = 16
warmup_ratio = 0.1

# Identify layers to prune with sensitivity consideration
def get_prunable_layers(model):
    layers_to_prune_list = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            # Only prune attention weights, not all linear layers
            if any(substr in name for substr in ['q_lin', 'k_lin', 'v_lin']):
                layers_to_prune_list.append((module, 'weight'))
    return layers_to_prune_list

> **Strategic layer selection**: We're targeting 50% sparsity but only in specific layers—the attention query, key, and value projections (q_lin, k_lin, v_lin). These layers are redundancy-rich but not critical for output, making them ideal pruning candidates. By leaving other layers untouched (like output projections and feed-forward networks), we preserve the model's core decision-making ability. 
> 
> Then, the gradual approach (100-step intervals over 10 epochs) gives the model ample time to adapt as weights disappear—like slowly removing training wheels rather than yanking them off!

## Step 5: Train the model with gradual pruning on a cubic schedule

Now we'll implement the heart of training-time pruning: gradually removing weights while the model learns to compensate. 

While Hugging Face's `Trainer` supports callbacks, we'll use a custom training loop for explicit control over the pruning process.

In [7]:
# Cubic sparsity schedule
def cubic_sparsity_schedule(step, total_steps, initial_sparsity=0.0, final_sparsity=0.5, start_step=0, end_step=None):
    """Cubic sparsity increase schedule (3-phase: slow-fast-slow)"""
    if end_step is None:
        end_step = total_steps
    
    if step < start_step:
        return initial_sparsity
    elif step >= end_step:
        return final_sparsity
    else:
        progress = (step - start_step) / (end_step - start_step)
        # Cubic function that starts and ends slowly
        sparsity_range = final_sparsity - initial_sparsity
        return initial_sparsity + sparsity_range * (3 * progress**2 - 2 * progress**3)
    
# Make magnitude pruning implementation to be non-cumulative
def apply_magnitude_pruning(model, layers_to_prune, target_sparsity):
    """Apply magnitude pruning to reach exact target sparsity"""
    # First, remove any existing pruning
    for module, param_name in layers_to_prune:
        if hasattr(module, param_name + '_mask'):
            prune.remove(module, param_name)
    
    # Apply fresh pruning to exact target
    prune.global_unstructured(
        layers_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=target_sparsity
    )
    
# Calculate sparsity
def calculate_sparsity(model, layers):
    """Calculate actual sparsity in pruned layers"""
    total_params = 0
    zero_params = 0
    for module, name in layers:
        if hasattr(module, name + '_mask'):
            mask = getattr(module, name + '_mask')
            weight = getattr(module, name + '_orig')
            current = weight * mask
        else:
            current = getattr(module, name)
        zero_params += torch.sum(current == 0).item()
        total_params += current.nelement()
    return zero_params / total_params if total_params > 0 else 0.0

# Evaluation during training
def evaluate_during_training(model, dataloader, compute_metrics_fn):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_loss += outputs.loss.item()
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            
            # Convert to lists instead of numpy arrays
            all_preds.extend(predictions.cpu().tolist())
            all_labels.extend(batch['labels'].cpu().tolist())
    
    # Pass the data in the expected format
    eval_pred = (all_preds, all_labels)
    metrics = compute_metrics_fn(eval_pred)
    model.train()
    return metrics, total_loss / len(dataloader)

> **Choosing the right schedule**: The cubic schedule follows an S-curve: slow→fast→slow. This mimics how we learn—starting cautiously, making rapid progress once comfortable, then fine-tuning at the end. For pruning, this means:
> - **Slow start (0-20%)**: Model learns which weights matter most
> - **Acceleration (20-80%)**: Rapid pruning while model is adaptable
> - **Slow finish (80-100%)**: Gentle final adjustments for stability
> 
> Alternative schedules like linear or exponential can work, but cubic often gives the best accuracy retention.

In [8]:
# Setup pruning on a fresh model
model_to_prune = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_to_prune = model_to_prune.to(device)
layers_to_prune = get_prunable_layers(model_to_prune)

print(f"Identified {len(layers_to_prune)} layers for pruning (reduced from 36)")
print("Layer types selected for pruning:")
for name, module in model_to_prune.named_modules():
    if any((module, 'weight') == layer for layer in layers_to_prune):
        print(f"  - {name}")
        if len([n for n, _ in model_to_prune.named_modules() if n.startswith(name)]) > 5:
            print("    ...")
            break

# Training setup with improved configuration
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
num_training_steps = num_epochs * len(train_dataloader)
num_warmup_steps = int(warmup_ratio * num_training_steps)

optimizer = AdamW(model_to_prune.parameters(), lr=learning_rate, weight_decay=0.01)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_warmup_steps, 
    num_training_steps=num_training_steps
)

# Initialize pruning infrastructure (0% to start)
prune.global_unstructured(
    layers_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.0
)

Identified 18 layers for pruning (reduced from 36)
Layer types selected for pruning:
  - distilbert.transformer.layer.0.attention.q_lin
  - distilbert.transformer.layer.0.attention.k_lin
  - distilbert.transformer.layer.0.attention.v_lin
  - distilbert.transformer.layer.1.attention.q_lin
  - distilbert.transformer.layer.1.attention.k_lin
  - distilbert.transformer.layer.1.attention.v_lin
  - distilbert.transformer.layer.2.attention.q_lin
  - distilbert.transformer.layer.2.attention.k_lin
  - distilbert.transformer.layer.2.attention.v_lin
  - distilbert.transformer.layer.3.attention.q_lin
  - distilbert.transformer.layer.3.attention.k_lin
  - distilbert.transformer.layer.3.attention.v_lin
  - distilbert.transformer.layer.4.attention.q_lin
  - distilbert.transformer.layer.4.attention.k_lin
  - distilbert.transformer.layer.4.attention.v_lin
  - distilbert.transformer.layer.5.attention.q_lin
  - distilbert.transformer.layer.5.attention.k_lin
  - distilbert.transformer.layer.5.attention.v_l

> **Pruning infrastructure setup**: Starting with 0% pruning might seem odd, but it's crucial for proper tracking. This initialization:
> - Creates weight masks for all target layers (initially all ones)
> - Separates original weights from their masks internally
> - Enables consistent sparsity calculation throughout training
> - Ensures the pruning machinery is ready before training begins
> 
> L1Unstructured pruning removes weights with smallest absolute values first—the assumption being that tiny weights contribute least to the model's decisions. Think of it as removing the quietest voices in a conversation first.

In [9]:
print("Starting gradual magnitude pruning with cubic schedule...")
model_to_prune.train()
global_step = 0
training_loss = []
eval_metrics_history = []
sparsity_history = []

# Pruning starts after warmup and ends before final epoch
pruning_start_step = num_warmup_steps
pruning_end_step = int(0.85 * num_training_steps)

for epoch in range(num_epochs):
    epoch_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in progress_bar:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model_to_prune(**batch)
        loss = outputs.loss
        epoch_loss += loss.item()
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_to_prune.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        global_step += 1
        
        # Apply gradual pruning with cubic schedule
        if global_step % pruning_frequency == 0 and global_step >= pruning_start_step:
            # Calculate current target sparsity using cubic schedule
            current_sparsity = cubic_sparsity_schedule(
                global_step, 
                num_training_steps, 
                initial_sparsity=0.0, 
                final_sparsity=target_sparsity,
                start_step=pruning_start_step,
                end_step=pruning_end_step
            )
            
            # Apply fresh pruning (not cumulative)
            apply_magnitude_pruning(model_to_prune, layers_to_prune, current_sparsity)
            
            # Track sparsity
            actual_sparsity = calculate_sparsity(model_to_prune, layers_to_prune)
            sparsity_history.append((global_step, actual_sparsity))
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'sparsity': f'{actual_sparsity*100:.1f}%',
                'target': f'{current_sparsity*100:.1f}%',
                'lr': f'{scheduler.get_last_lr()[0]:.2e}'
            })
    
    # End of epoch evaluation
    training_loss.append(epoch_loss / len(train_dataloader))
    eval_dataloader = DataLoader(eval_dataset, batch_size=32)
    eval_metrics, eval_loss = evaluate_during_training(model_to_prune, eval_dataloader, compute_metrics)
    eval_metrics_history.append(eval_metrics)
    
    print(f"Epoch {epoch+1}: Train Loss={training_loss[-1]:.4f}, "
          f"Eval Loss={eval_loss:.4f}, "
          f"Eval Acc={eval_metrics['accuracy']:.4f}, "
          f"Eval F1={eval_metrics['f1']:.4f}, "
          f"LR={scheduler.get_last_lr()[0]:.2e}")

# Apply final pruning to reach exact target
apply_magnitude_pruning(model_to_prune, layers_to_prune, target_sparsity)

# Calculate final model sparsity
final_sparsity = calculate_sparsity(model_to_prune, layers_to_prune)
print(f"\nTraining complete. Final sparsity: {final_sparsity*100:.2f}%")

Starting gradual magnitude pruning with cubic schedule...


Epoch 1/10: 100%|██████████| 125/125 [00:19<00:00,  6.42it/s]


Epoch 1: Train Loss=0.0613, Eval Loss=0.4506, Eval Acc=0.8900, Eval F1=0.8900, LR=1.00e-05


Epoch 2/10: 100%|██████████| 125/125 [00:19<00:00,  6.32it/s, loss=0.0003, sparsity=0.9%, target=0.9%, lr=9.89e-06]


Epoch 2: Train Loss=0.0362, Eval Loss=0.5106, Eval Acc=0.8950, Eval F1=0.8949, LR=9.70e-06


...

Epoch 8: Train Loss=0.0020, Eval Loss=0.6613, Eval Acc=0.9025, Eval F1=0.9023, LR=1.17e-06


Epoch 9/10: 100%|██████████| 125/125 [00:20<00:00,  5.98it/s, loss=0.0002, sparsity=50.0%, target=50.0%, lr=4.32e-07]


Epoch 9: Train Loss=0.0012, Eval Loss=0.6568, Eval Acc=0.9050, Eval F1=0.9048, LR=3.02e-07


Epoch 10/10: 100%|██████████| 125/125 [00:20<00:00,  5.97it/s, loss=0.0001, sparsity=50.0%, target=50.0%, lr=4.87e-08]


Epoch 10: Train Loss=0.0013, Eval Loss=0.6575, Eval Acc=0.9050, Eval F1=0.9048, LR=0.00e+00

Training complete. Final sparsity: 50.00%


> **Training dynamics analysis**: Watch how the model adapted during pruning:
> - _**Accuracy resilience**_: Despite removing 50% of attention weights, accuracy stayed around 90%
> - _**Loss fluctuations**_: Small spikes when pruning accelerated (epochs 4-7) followed by recovery
> - _**Learning rate decay**_: Cosine schedule helped stabilize training as pruning intensified
> - _**Final convergence**_: Model stabilized at target sparsity with minimal accuracy loss
> 
> For production, consider: longer training, larger warmup period, or different learning rates for pruned vs unpruned layers.

## Step 6: Make pruning permanent

After fine-tuning with pruning masks, we need to make these changes permanent in the model's weight tensors.

In [10]:
# Make pruning permanent
print("Making pruning permanent for magnitude pruned model...")
for module, param_name in layers_to_prune:
    if hasattr(module, param_name + '_mask'):
        prune.remove(module, param_name)

Making pruning permanent for magnitude pruned model...


> **Permanent weight removal**: The prune.remove() operation converts temporary masks into permanent zero weights.
> 
> Note that file size doesn't change for unstructured pruning—the model structure remains the same. The real benefits come from reduced FLOPs during inference, especially on hardware supporting sparse operations.

## Step 7: Evaluate model performance

Finally, let's evaluate model performance and compare against baseline.

In [11]:
# Calculate final statistics
pruned_size = get_model_size_mb(model_to_prune)
print(f"Pruned model size: {pruned_size:.2f} MB")

# Calculate final sparsity after removal
def calculate_final_sparsity_post_remove(model, layers_info):
    total_params = 0
    zero_params = 0
    for module, name in layers_info:
        weight = getattr(module, name)
        total_params += weight.nelement()
        zero_params += torch.sum(weight == 0).item()
    return float(zero_params) / total_params if total_params > 0 else 0.0

final_sparsity_after_remove = calculate_final_sparsity_post_remove(model_to_prune, layers_to_prune)
print(f"Final sparsity of pruned layers: {final_sparsity_after_remove*100:.2f}%")

# Full evaluation of the pruned model
trainer_pruned = Trainer(
    model=model_to_prune,
    args=TrainingArguments(
        output_dir=f"{output_dir}/pruned_eval",
        per_device_eval_batch_size=32,
        do_train=False,
        do_eval=True,
        report_to=[]
    ),
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("\nEvaluating final pruned model...")
pruned_results = trainer_pruned.evaluate()
print(f"Final metrics: Accuracy={pruned_results['eval_accuracy']:.4f}, F1={pruned_results['eval_f1']:.4f}")

# Compare inference speeds
eval_dataloader = DataLoader(eval_dataset, batch_size=1)  # Single sample for latency
baseline_time, baseline_std = measure_inference_speed(baseline_model, eval_dataloader)
pruned_time, pruned_std = measure_inference_speed(model_to_prune, eval_dataloader)

speedup = (baseline_time - pruned_time) / baseline_time * 100
print(f"\nInference speed comparison:")
print(f"Baseline: {baseline_time*1000:.2f} ± {baseline_std*1000:.2f} ms")
print(f"Pruned: {pruned_time*1000:.2f} ± {pruned_std*1000:.2f} ms")
print(f"Speedup: {speedup:.1f}%")

Pruned model size: 255.42 MB
Final sparsity of pruned layers: 50.00%


/tmp/ipykernel_76/4065736787.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_pruned = Trainer(



Evaluating final pruned model...


Final metrics: Accuracy=0.9050, F1=0.9048

Inference speed comparison:
Baseline: 5.24 ± 0.14 ms
Pruned: 5.32 ± 0.19 ms
Speedup: -1.5%


> **Why no speedup?** This is completely expected! Unstructured pruning creates randomly scattered zeros throughout weight matrices, but standard hardware (CPUs/GPUs) can't skip these zeros efficiently. They still load the full matrix and multiply by zero. The slight slowdown (-1.9%) comes from overhead in applying masks during inference.
>
> **Getting actual speedups requires**:
> - Specialized hardware with sparse tensor cores (NVIDIA A100, Apple M1 Neural Engine)
> - Sparse-aware inference libraries (DeepSparse, TensorRT, ONNX Runtime)
> - Converting to true sparse format (CSR, CSC matrices)
>
> In production, you could export to ONNX and use sparse inference engines to realize the speed benefits.

## Conclusion

Gradual magnitude pruning during training demonstrates the power of adaptation—by removing weights progressively while the model learns, we achieved 90.2% accuracy at 50% sparsity, significantly outperforming post-training methods. 

This technique showcases how training-time compression allows models to reorganize and compensate for removed connections, maintaining performance while reducing redundancy.

In [15]:
# Final summary and recommendations
print("=" * 60)
print("GRADUAL MAGNITUDE PRUNING RESULTS SUMMARY")
print("=" * 60)

print(f"\nBaseline Model:")
print(f"  Accuracy: {baseline_results['eval_accuracy']*100:.1f}%")
print(f"  F1 Score: {baseline_results['eval_f1']*100:.1f}%")
print(f"  Size: {baseline_size:.2f} MB")
print(f"  Parameters: {total_params:,}")

print(f"\nPruned Model ({target_sparsity*100}% sparsity):")
print(f"  Accuracy: {pruned_results['eval_accuracy']*100:.1f}%")
print(f"  F1 Score: {pruned_results['eval_f1']*100:.1f}%")
print(f"  Size: {pruned_size:.2f} MB (no reduction due to unstructured nature)")
print(f"  Actual Sparsity: {final_sparsity_after_remove*100:.1f}%")

print(f"\nPerformance Impact:")
accuracy_change = pruned_results['eval_accuracy'] - baseline_results['eval_accuracy']
if accuracy_change > 0:
    print(f"  Accuracy change: +{accuracy_change*100:.1f}% (improved!)")
else:
    print(f"  Accuracy retained: {100 + accuracy_change*100:.1f}%")
print(f"  Inference speedup: {speedup:.1f}% (expected on standard hardware)")
print(f"  Meets 90% accuracy requirement: {'✓' if pruned_results['eval_accuracy'] > 0.90 else '✗'}")

print("\n" + "=" * 60)
print("TRAINING-TIME vs POST-TRAINING COMPARISON")
print("=" * 60)

print("\nPost-training pruning (Lesson 2):")
print("  • 30% sparsity → 84% accuracy (-6% from baseline)")
print("  • 50% sparsity → ~78% accuracy (estimated)")
print("  • Immediate application, no retraining needed")
print("  • Limited ability to recover performance")

print("\nGradual magnitude pruning (this demo):")
print(f"  • 50% sparsity → {pruned_results['eval_accuracy']*100:.1f}% accuracy (minimal loss!)")
print(f"  • Requires training time but achieves better compression-accuracy tradeoff")
print(f"  • Model adapts during pruning, redistributing importance")

print(f"\n🎯 Key Insight: At 50% sparsity, training-time pruning achieved {pruned_results['eval_accuracy']*100:.1f}%")
print(f"   vs estimated ~78% for post-training methods—a {(pruned_results['eval_accuracy']*100 - 78):.1f}% advantage!")

print("\n" + "=" * 60)
print("UNDERSTANDING DIFFERENT PRUNING APPROACHES")
print("=" * 60)

print("\nUnstructured Pruning (what we did):")
print("  ✓ Removes individual weights")
print("  ✓ Achieves high sparsity with good accuracy")
print("  ✗ No immediate speedup on standard hardware")
print("  → Best for: Research, sparse-aware deployment")

print("\nStructured Pruning (alternative approach):")
print("  ✓ Removes entire channels/neurons/heads")
print("  ✓ Immediate speedup on all hardware")
print("  ✗ Typically lower accuracy at same sparsity")
print("  → Best for: Standard hardware deployment")

print("\n" + "=" * 60)
print("DEPLOYMENT STRATEGIES")
print("=" * 60)

deployment_options = [
    "1. **Sparse-aware inference**: Use DeepSparse or TensorRT for actual speedups",
    "2. **Hardware selection**: Deploy on devices with sparse tensor support",
    "3. **Format conversion**: Export to ONNX with sparse operations enabled",
    "4. **Hybrid approach**: Combine with int8 quantization for additional benefits",
    "5. **Architecture swap**: Consider structured pruning if hardware lacks sparse support"
]

for option in deployment_options:
    print(option)

print("\n" + "=" * 60)
print("KEY TAKEAWAYS")
print("=" * 60)

takeaways = [
    "• Training-time pruning >> post-training pruning for accuracy retention",
    "• Gradual pruning with proper scheduling is crucial for success",
    "• Hardware compatibility determines whether you see actual speedups",
    "• Unstructured pruning is great for research, structured for deployment",
    "• Always validate on your target hardware before committing to an approach"
]

for takeaway in takeaways:
    print(takeaway)

print("\n" + "=" * 60)
print("NEXT STEPS FOR YOUR EDGE DEPLOYMENT")
print("=" * 60)

next_steps = [
    "1. Experiment with different pruning schedules:",
    "   • Try linear vs cubic vs exponential schedules",
    "   • Adjust pruning start/end points in training",
    "   • Test different target sparsity levels",
    "",
    "2. Visualize weight distributions and sparsity patterns:",
    "   • Create heatmaps showing which layers are most sparse",
    "   • Plot weight magnitude distributions before/after pruning",
    "   • Analyze attention head importance across layers",
    "",
    "3. Explore pruning impact on model behavior:",
    "   • Test performance on specific sentiment categories",
    "   • Identify which review types are most affected",
    "   • Analyze failure cases and edge examples",
    "",
    "4. Export the sparse model for deployment:",
    "   • Convert to ONNX format with sparse operators",
    "   • Test with DeepSparse or TensorRT inference engines",
    "   • Benchmark on actual ARM edge devices",
    "   • Integrate with existing application pipeline"
]

for step in next_steps:
    print(step)

print("\n✨ Remember: Choose the compression technique that matches your")
print("   deployment constraints, not just the one with best paper results!")
print("=" * 60)

GRADUAL MAGNITUDE PRUNING RESULTS SUMMARY

Baseline Model:
  Accuracy: 90.2%
  F1 Score: 90.2%
  Size: 255.42 MB
  Parameters: 66,955,010

Pruned Model (50.0% sparsity):
  Accuracy: 90.5%
  F1 Score: 90.5%
  Size: 255.42 MB (no reduction due to unstructured nature)
  Actual Sparsity: 50.0%

Performance Impact:
  Accuracy change: +0.3% (improved!)
  Inference speedup: -1.5% (expected on standard hardware)
  Meets 90% accuracy requirement: ✓

TRAINING-TIME vs POST-TRAINING COMPARISON

Post-training pruning (Lesson 2):
  • 30% sparsity → 84% accuracy (-6% from baseline)
  • 50% sparsity → ~78% accuracy (estimated)
  • Immediate application, no retraining needed
  • Limited ability to recover performance

Gradual magnitude pruning (this demo):
  • 50% sparsity → 90.5% accuracy (minimal loss!)
  • Requires training time but achieves better compression-accuracy tradeoff
  • Model adapts during pruning, redistributing importance

🎯 Key Insight: At 50% sparsity, training-time pruning achieved 